<a href="https://colab.research.google.com/github/NamanJhawar-NMIMS/Project-Search-Engine/blob/main/Project_Search_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np